# Discriminative model VS generative model
Discriminative model: used for predictions based on input

Generative model: generate a set of features (output) with the noise and sometimes a class as inputs

The noise is used for the model to generate diverse features

# GANs
Generative Adversarial Networks (GANs): GANs is a neural network architecture that are used for generating new examples for image datasets in various domains

GANs consists two neural network, the generator and the discriminator. They are trained simultaneously through adversarial training. The geneator takes random noise as input and try to produce realistic data (eg. images), while the discriminator takes in both real data and the data produced by the generator and trying to distinguish which one is real

During training, the generator tries to produce data that the discriminator can’t distinguish from real data, while the discriminator tries to get better at differentiating real data from fake data. The two networks are in essence competing in a game, so both networks improves at the same time

At the end, the generator will be able to produce realistic images that the discriminator cannot distingish from real data

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*20eOTTCwC-jGXpbAoc9yVg.png" width=700>

## Discriminator
The discriminator is a binary classsifier that trys to predict the probability of an input data being real or fake, where

$$output = P(Y|X)$$

$P(Y|X)$: the probability (between 0 and 1) of an input being real/fake. An probability of 0 indicates that the input data is 100% fake and 1 indicates that the input data is 100% real

$X$: the input data, can be real images or generated images by the generator

$Y$: the class categories (real or fake in this case)

In general, the architecture of the discriminator is the same as the traditional CNN with the following layers: convolutional layers, batch norm layers, pooling layers, activations functions, fully connected layers, and the output layer

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*e0Dnwo9V92Tjkv_PAJj5fQ.png" width=700>

### Binary cross entropy loss (BCE)
The discriminator is a binary classificer since it only predicts whether the given input is real or fake, so the loss funtion is the traditional BCE loss function, given by

$$ J(\theta) = -\frac{1}{m} \sum_{i=0}^{m-1} \left[ (y^{(i)}) \log\left(\hat {y^{(i)}}) \right) + \left( 1 - y^{(i)}\right) \log \left( 1 - \hat {y^{(i)}}) \right) \right]$$

$J(\theta)$: the average loss over all training examples within a batch

$\theta$: the parameters of the discriminator

$m$: the number of training examples in a batch

$y^{(i)}$: the true label of the $i$th training example (real/fake)

$\hat {y^{(i)}}$: the label of the $i$th training example predicted by the discriminator. It depends on the input features of each training example, $x^{(i)}$, and the parameters of the discriminator, $\theta$

## Generator
The generator will take in a random noise vector, $\xi$, to generate a set of features, $\hat {X}$, that will produce a realistic image (Different noies vectors ensure the outputs are diverse). Then, the output, $\hat {X}$, will be the input to the discriminator, and the parameters of the generator will be updated based on the prediction given by the discriminator

The generator also uses the BCE loss function, and during calculation, all generated data will have labels of 1, so the input to the BCE loss function are a tensor with value predicted by the discriminator and a label tensor with all values equal to 1. This tells the generator how far it is for the generated data to real data (We only label generated data with 1 when training the generator, otherwise it would confuse the discriminator)

During training, the generator will learn to capture the distribution of data, so more commonly seen data will have a higher probability of being generated


## GAN training
In GAN training, the process is alternating, meaning only one network (whether the generator or the discriminator) will be updated everytime. Also, the training requires both networks to have similar skill level without one network significantly outperforming the other; otherwise, the superior network will always give very accurate perdictions and pervent the other network to learn

Process:

1. Input some random noises, $\xi$, to the generator and output some features, $\hat {X}$

2. Combine the generated features, $\hat {X}$, and real features, $X$, and input them into the discriminator

3. Let the discriminator makes the predictions, $\hat Y$, and use the predictions to compute cost and update the discriminator's parameters (Only update the discriminator)

4. Input some random noises to the generator and output some features

5. Only input the generated features into the discriminator to make predictions

6. Calculate the cost with all labels of the generated features to be one, this shows how far the generated features are away from real examples

7. Use the cost to update the generator parameters (Only update the generator)


# Activation functions
Activation functions must be differentiable and non-linear. Differentiable ensures the network can learn from backpropagation; non-linear ensures the network can learn a complex pattern (otherwise, any network with linear activations can be collapsed into a simple linear regression model)

## Common activation functions
1. ReLU
2. Leaky ReLU
3. Sigmoid
4. Tanh

## Dying ReLU problem
When most of the neurons takes in negative input and output zero, the gradient of the ReLU function is always 0, so the weights stop getting updated and the network stops learning. Once it becomes dead, it is impossible to recover the network to learn

Leaky ReLU can effectively solve this problem

## Vanishing gradient
When the inputs to the sigmoid and tanh activation function are large (both positive and negative), the gradients used to update the weights during backpropagation is very close to 0. This makes it hard for the network to learn because the weights of the earlier layers change, which slows down or stops training

This can be solved using normalization, gradient clipping, etc


# Batch norm
When features have significantly different distribution, it can be difficult and slow for the network to learn and perform gradient descent. Batch normalization performs normalization at each hidden layer to smooths the cost function and reduce the internal covariate shift to speed up training and provide more stable performance

Process:
1. Take the activation, $z$, from the previous layer and normalize it to $z_{norm}$ have mean of 0 and standard deviation of 1
2. Set $z_{out} = \gamma z_{norm} + \beta$. $\gamma$ and $\beta$ are shift factors that control the mean and standard deviation of the output, $z_{out}$. These are parameters that are learned during training, which means the output, $z_{out}$, does not need to have a fixed mean of 0 and standard deviation of 1; rather, the most optimal mean and standard deviation values for the data set are adjusted and learned during the training

During testing, we will use the parameters learned from the training to make the distribution of the test data as similar to the training data as possible to improve the models' performance 

Detailed implemetation of batch norm: https://github.com/guaaaaa/My_ML_Notebook/blob/master/Neural%20network.ipynb

# Transpose convolution
Transpose convolution is used to upsample the feature maps by using a filter. The parameters of the filter can be learned during training. 

One issue with transpose convolution is that some pixels will be the sum from multiple filters, which can result in a checkerboard pattern.

<img src="https://www.jeremyjordan.me/content/images/2018/05/Screen-Shot-2018-05-19-at-3.12.51-PM.png" width=800>

# Mode collapse
In GANs, mode collapse happens when the generator focuses on only producing limited set of data that can decieves the discriminator but does not captures the full diversity of the data distribution

For example, in a GAN that can generate hand written digits (from 0 to 9). The feature space should have 10 local minima with each minima represents the peak distribution of each digit; this is called a mode

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*XVzW4smMmN6WZpBTrCMldA.png" width=700>

However, as the training progresses, the generator may find the generated digit 1 can fool the discriminator more easily than any other generated digits, so the generator will only generator 1 but not other characters, which means the generator converges to a single local minimum, and it gets stuck producing other mode of data


<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*iYSy1UlFtGcpWRFquJiqPA.png" width=700>


# Issue with BCE loss function
Mode collape is primarily caused by the BCE loss function. The goal of the generator is to model a similar distribution as the real data. However, during the training process, the discriminator's job is easier than the generator's job and will learn faster, so as the training progresses, the discriminator will likely to output a value very close to 0 or 1. This results in vanishing gradient problem, so the generator does not know how to improve and get stuck at a single local minimum

# Wasserstein loss
The W-loss function calculate the loss by using Wasserstein distance. The W-distance is the minimum cost of converting the generated distribution, $g$, into the real distribution, $r$.

<img src="https://www.researchgate.net/publication/349704621/figure/fig1/AS:996752224161799@1614655657970/Schematic-of-the-L-2-Wasserstein-distance-We-here-consider-optimal-transport-from-the.png" width=500>

W-loss function

$$loss = \mathbb{E}(c(X)) - \mathbb{E}(c(\hat X))$$

$\mathbb{E}$: the expected value (average)

$c$: the critic, which is equivalent to the discriminator but can output any real number while the discriminator only output value between 0 and 1

$X, \hat X$: the features of a real example and the features of a generated example respectively

The critics tries to maximize the differences between the two expected values to distinguish the real and generated example. The generator tries to minimize the differences to make the generated features close to the real features

Compared to the BCE loss, which only outputs a number between 0 and 1, the W-loss function can output any real number. This helps with the vanishing gradient problem and mode collapse

## 1-Lipschitz continuous
To use the W-loss, the critic need to be 1-L continuous, which means the gradient of the loss function must be a value between -1 and 1. This means that the W-loss function grows linearly to maintain the stability during training

To enforce 1-L continuous, we can use

1. Weight clipping: force the weights of the critics to be a specific interval of values. However, weight clipping can limit the performance of the critics

2. Gradient penalty: a softer constrains to encourage 1-L continuity by adding a regularization term that penalize the critic when its gradient is greater than 1, so the loss function become

$$loss = \mathbb{E}(c(r)) - \mathbb{E}(c(g)) + \lambda_{reg}$$

It's impossible to check the gradient of every single point on the loss function, so we randomly interpolate an images, $\hat X$,by combining a generated image and a real images with the factor $\epsilon$, where

$$\hat X = \epsilon r + (1 - \epsilon) g$$

$\hat X$: randomly interpolated image

$\epsilon$: a real number between 0 and 1

$r, g$: the real and generated image respectively

Thus, the regularization term is

$$\lambda_{reg} = \lambda \mathbb{E}(||\nabla c(\hat X)||_2 - 1)^2$$

$\lambda$: penalty coefficient

$||\nabla c(\hat X)||_2$: the L-2 norm of the gradient of critic on the interpolated image

The given function encourges the model to have gradient of 1 since when the gradient is one, the additional cost if 0. If the gradient is significantly larger than 1, the regularization term will be large to penalize the critic

# Conditional generation
Conditional GAN is a type of GAN that takes in a condition (class) as input and generates based on the given condition. This means that the model should be trained on labeled data

In addition to a noise vector, the generator also takes in a one-hot vector that contains the information of the desired class. Two vectors will be concatenate into one large vector as the input. The goal of the generator is to generate an output that looks realistic and matches the input class

Similarly, the discriminator will take in an image and a class label. It will only predict real if the input image is realistic and matches the class label. To train the discriminator, we use labeled dataset like training a multiclass classification model

To pass in the class information into the discriminator, we can use one-hot matrices where the number of matrices is the same as total number of class. Only the matrix that represents the class will be all 1, other matrices will be filled with 0. Alternatively, we can convert the label into an embedding and concatenate it to the images, which is a more compact representation

### **Steps for Training (WGAN)**

1. Prepare Inputs:
   - Sample a batch of real images and their labels from the dataset.
   - Generate random noise for the generator input.

2. Generate Fake Images:
   - Pass the noise and labels to the generator.
   - The generator produces fake images conditioned on the labels.

3. Critic Evaluation:
   - Pass both:
     - Real images concatenated with their labels.
     - Fake images concatenated with their labels.
   - The critic outputs scores indicating how real the images are and whether they match the labels.

4. Critic Loss:
   - Compute the critic's loss, which:
     - Encourages high scores for real images with correct labels.
     - Assigns low scores to fake images or real images with mismatched labels.
   - This helps the critic learn to detect label-image mismatches early in training when the generator's outputs are poor.

5. Update Critic:
   - Update the critic's weights based on the loss.

6. Generate New Fake Images:
   - Sample new noise.
   - Pass the noise and labels to the generator to produce another batch of fake images.

7. Generator Loss:
   - Pass the fake images and labels to the critic.
   - Compute the generator's loss, which:
     - Encourages the generator to produce images that the critic assigns high scores to.
     - Penalizes images that look fake or don't match the label.

8. **Update Generator:**
   - Update the generator's weights based on the loss.
   
The critic takes in the labels, convert it into an embedding, and concat it to the images channel. The generator takes in the label and convert it into the vector and concat it to the noise.

The critic learn to tell whether the images are real and if the images matches the label, so, even the generated images looks real but does not matches the label, the critic will give it a high loss. The key is the embedding is just an additional feature that guilds the discriminator and generator, where we only consider the images that looks real and matches the label to be "real"


# Controllable generation
Controllable generation allows the user to control the generated feature (eg. hair colour, age, etc). This is done by tweaking the input noise vector. Compared to conditional generation, controllable generation does not need a labeled training set or class information (it's only controlled by the input noise vector)

# Feature manipulations
## Z space
Z space is the space of all noise vectors. In the Z space, two vectors close together will have similar features, and two vectors that are further apart will have different features

To find the vector that controls a specific feature, we find the two vector that gives the exact same output except the wanted feature (eg. the same woman, one with red hair and one with blue hair). The vector that can turns the features of hair colour from red to blue is given by the difference of that two noise vectors. By adding this vector to any noise vector that outputs someone with red hair, the output will be the same person but with blue hair. The z latent space continuous, which allows interpolation between images

## Challenges
1. Correlation of features: some features are likely to have strong correlations (like beard and masculinity), so changes on one feature (adding beard) can cause changes on other features (face become more masculine) even that's not the intention

2. Vector space entanglement: one entry in the noise vector can impact multiple features, even those features may not have actual correlations. This is because the dimension of the noise vector is too small comparing to the number of features we want to control, so the noise vector cannot have a one to one correlation with the features. For exmample, if we want to control 100 features using a vector of size 5, it's impossible to have individual control on every single feature

<img src="https://miro.medium.com/v2/resize:fit:601/1*JcdGoZU6hFSnK4JghcJRxw.png">

# Classifier gradient
To find a specific vector in the Z space that can control a specific feature, we feed some noise to the generator to get some outputs. Then, we feed the outputs to into a pre-trained classifier to detect whether the generated outputs have the features we wanted. Based on the prediction, we update the noise vector until the generator can generate the feature we wanted

Note: the process is done after training the generator, so the weights in the generator should not be modified

# Disentanglement
With a disentangled Z space, we can change on feature in the output without changing other features. In general, a disentangled Z space requires more entries in the noise vector than the number of features we want to control

Latent factors: the components of the noise vector that can impact the output features, but cannot be directly observed

# Evaluation
Evaluation of a GAN model can be difficult since there's no ground truth to check with. Therefore, we use fidelity and diversity as two important metrics to evaluate a GAN

* Fidelity: the quality of each generated image (how realistics the generated images are)

* Diversity: the distribution of generated images (does the model captures the real distribution of data)

## Comparing images
To compare how similar two images are, we can use their feature distance by extracting higher level features of the images and then compare. This is a more reliable way than pixel distance

To extract features from an image, we use a pre-trained classifier (eg. Inception-v3) by passing in the image as input and extract the embedding / feature map from a middle layer. This embedding contains information about the features present in the input image. An embedding extracted from a later layer has more higher level features, where an extraction from a ealier layer has more lower level features. The extracted embedding is given by $\Phi (X)$, where $\Phi$ is a function based on the selected pre-trained classifier and $X$ is the input image

Then, we can compare two images based on their embedding

## Frechet inception distance (FID)
FID is a metric that calculates the feature distance between the real and generated images by comparing their feature distributions using a pre-trained network. Lower scores indicate the two groups of images are more similar, a 0 score indicates the two images are identical

### Covariant matrix
Covariance is a measure of the joint variability of two random variables; it indicates how the two variables vary together

A Convariant matrix represents the covariance values of each pair of variables in multivariate data; along the diagonal, the value represents the variance of each variable. If a non-diagonal entry is 0, it means that the two variables are independent from each other. In our case, each variable represents a feature, so the covariant matrix, $\Sigma$, measures the distribution of each feature within a set (eg. a group of real/fake images), and how each feature is related to another feature

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*J6z7xcleH9wxHGGCLvDptg.jpeg" width=700>

### FID formula
Given two images / two groups of images, $X, Y$, and their covariant matrices, $\Sigma_X, \Sigma_Y$, the FID is calculated by

$$FID = ||\mu_X - \mu_Y||^2 + Tr(\Sigma_X + \Sigma_Y - 2\sqrt{\Sigma_X\Sigma_Y})$$

$\mu_X, \mu_Y$: the mean of $X$ and $Y$; in multivariable case, both $X$ and $Y$ are vectors with each entry representing the mean of a single feature

$Tr$: the trace of a matrix (sum of all diagonal elements)

$\Sigma_X, \Sigma_Y$: the covariant matrices of image $X$ and $Y$

On an entire dataset, we first extract the features for both real and generated images. Then, we compute 1 mean and 1 covariance matrix for the entire set of real images and another for the generated images to compute the FID score. FID provides a global comparison of the real and generated datasets, computing multiple means or covariances for different subsets of images will make the computation more complicated

If the dataset is too large, it's common to select a subset of representative data from the entire dataset to perform FID calculations. Also, if we want to class-wise performance, we need to compute FID scores separately for each class

### FID limitations
* FID uses a pre-trained Inception model trained on ImageNet, which may not capture key features in all cases
* Need large sample size
* Slow to run
* Use limited statistics (only mean and covariance)
* FID is sensitive to mode collapse in GANs. Even if the generator produces a few high-quality samples, a poor variety of generated samples can lead to a high FID score.

## Inception score (IS)
Inception score is another method for evaluating GANs. First, we feed some generated images into the Inception model, but instead of extracting the features from a middle layer, we uses the classifier output to perform statistics

* Fidelity (low entropy): the fidelity of a GAN is measured by $p(y|x)$, which means the classifier should be confident which class the input images belongs to. So given a generated image, $x$, the prediction on a class $y$ should be high, indicating a defined, high peak

* Diversity (high entrpoy): the diversity of a GAN is measured by $p(y)$, which means the GAN should be able to generate all classes in the set instead of only generating a subset

Thus, idealy, a good GAN should be model the distribution with the number of peaks equals the number of total number of classes (high diversity) with each peak being high and defined (high fidelity)

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*xLq69Oj6_4NRZoWleg9K_Q.png">

### KL divergence
KL divergence measures how different two given distributions are. KL divergence is high when our distributions are dissimilar. That is, it is high when each generated image has a distinct label and the overall set of generated images has a diverse range of labels. Ideally, the label distribution and the margin distribution should be vastly different  

KL divergence is calculated by

$$D_{KL}(p(y|x)||p(y)) = p(y|x) \log(\frac{p(y|x)}{p(y)})$$



The IS score is then calcualted by

$$IS = exp(\mathbb{E}_{x \sim p} D_{KL}(p(y|x)||p(y)))$$

In general, a higher IS score indicates a better GAN

### Limitations
1. If the generator generates only one image per class, repeating each image many times, it can score highly
2. The IS score only looks at generated images but not the real images
3. If the generator memorizes the training data and replicates it, it can score highly
3. The IS score uses a pre-trained Inception model trained on ImageNet, which may not capture key features in all cases (eg. generate a class that's not in the Inception network dataset can lead to always lot IS score)

## Sampling
When evaluating GAN, the real images can be sampled using the normal distribution of data. For fake images, the images being sampled can greatly impact the evaluation result of GAN. Therefore, we use the truncation trick to ensures we only samples from the distribution we want.

Truncation trick will only sample noise vectors within a distance, $d$, from the mean of the distribution, where $d$ is a hyperparameters. Higher value of $d$ means a larger area of the distribution is covered, making the samples to have higher diversity but lower fidelity. Lower value of $d$ means a smaller area of the distribution is covered, making the samples to have lower diversity but higher fidelity

However, the FID may not works well with the truncation trick since FID requires the sample to have both high diversity and fidelity, but the truncation trick focus more on one of those aspect. Therefore, the method of evaluation highly depends on the application of the model, and human evaluation is also an crucial part

## Precision and recall for GAN
$P_r$ represents the distribution of the real samples, and $P_g$ represents the distribution of the generated samples. Ideally, we want the generated distribution to be the exact same as the real distribution (not a subset or super set of it)

* Precision (Fidelity)
$$Precision = \frac{P_r \bigcup P_g}{P_g}$$

Precision represents the percentage of the generated examples that look real. Higher precision indicates higher fidelity, meaning more generated examples will look real. However, a 100% percentage does not mean the GAN is perfect since $P_g$ can be a subset of $P_r$ and does not fully capture and real distribution

* Recall (Diversity)
$$Recall = \frac{P_r \bigcup P_g}{P_r}$$

Recall represents the percentage of the real distribution that is covered by the generated distribution. Higher recall indicates higher diveristy, meaning the generated distribution can capture more real distribution. However, a 100% recall may occur when $P_g$ is a superset of $P_r$. This means the generator can generate all real distribution, but also things that does not look real  (The truncation trick can help to resolve this issue)

<img src="https://www.researchgate.net/publication/350132222/figure/fig9/AS:1002549385629711@1616037808862/a-Illustration-of-precision-recall-for-distribution-of-real-images-Pr-blue-and-the.ppm">

# Disadvantage of GANs
1. Lack of instrinsic evaluation metrics (require inspections on many generate samples)
2. Unstable and slow training process
3. No density estimation (it is very difficult to estimate the distribution of the generator, like how often would a specific feature be generated)
4. Invertion process (taking in an image and output put its noise vector) is difficult. Invertion can be helpful to build better controllable GAN

# StyleGANs
GANs have improved in
1. Stability: longer, more stable training that can generate better results
2. Capacity: increasing model size and generating high resolution images
3. Diversity: increasing variety of the generated images

StyleGAN is a state-of-the-art GAN that achieves high fidelity (resolution), high diversity, and have more control over detailed image features.

The main components of StyleGAN includes:
1. Progressive growing
2. Noise mapping network
3. Adaptive instance normalization (AdalN)

## Progressive growing
A common issue of traditional GANs is the output image size are often very small to ensures training stability (eg. 64x64). Generating high resolution images is challenging because the generator must learn how to output both large structure and fine details at the same time; having wrong details can make the generated images to be easily classified as fake. Also, training for generating high resolution images require significantly more memory and time

Progressive growing is a solution that helps the generator to have stable training when generating high resolution images. It involves starting with a very small image and incrementally adding blocks of layers that increase the output size of the generator model and the input size of the discriminator model to increase resolution. This incremental nature allows the training to first discover large-scale structure of the image distribution and then shift attention to increasingly finer scale detail, instead of having to learn all scales simultaneously.

<img src="https://www.researchgate.net/publication/334404121/figure/fig2/AS:779799152844802@1562930014736/Details-of-the-architecture-of-the-progressive-growing-network-According-to-the-training.png" width=700>

Instead of suddenly insert new layers to the networks, we fade them in gradually using a blending factor $\alpha$ to prevent sudden shocks to the already well-trained, smaller-resolution layers. Also, all layers remain trainable during the training process, including existing layers when new layers are added

<img src="https://i0.wp.com/theailearner.com/wp-content/uploads/2019/11/transition-prog-gan.jpg?resize=625%2C326&ssl=1" width=500>


### Growing the Generator
When inserting a new layer to the generator, we need 2 layers in parallel, an upsampling layer, which simply upsamples the image size by 2 without any learnable parameters, and an normal convolutional block that doubles the output image size, which contains both conv and activations layer inside. Essentially, both layers will have the same output dimension, which doubles the image sizes from the previous layer

The output from the upsampling layer and convolutional block is combined using a **blending factor** $\alpha$, which transitions smoothly from 0 to 1 over time. This blending process ensures a gradual shift in the generator's behavior, preventing destabilization during training.

The output from these layers is combined using a blending factor, $\alpha$, which gradually increases from 0 to 1 over time. This blending process is defined as:

$$
\text{Output} = (1 - \alpha) \cdot \text{Upsampling Output} + \alpha \cdot \text{Conv Block Output}
$$

* Initially, $\alpha = 0$, so the output depends entirely on the upsampling layer.
* As $\alpha$ gradually increases, the contribution of the convolutional block grows while the upsampling layer’s influence diminishes.
* When $\alpha = 1$, the output depends entirely on the convolutional block.

This gradual transition minimizes the risk of sudden shifts in the learned distributions when a new layer is added to ensure stable training. Once the blending process for one resolution is complete (when $\alpha = 1$), the next layer can be added to further increase the image resolution

### Growing the Discriminator
Similar to growing the generator, the discriminator is grown by having 2 layers in parallel, an downsampling layer or pooling layer, which downsamples the image size by 2 without any learnable parameters, and a normal convolution block. Again, both layers will have the same output dimensions, which reduces the input image size by 2

The output is also blended with the blending factor $\alpha$, following the formula:

$$
\text{Output} = (1 - \alpha) \cdot \text{Downsampling Output} + \alpha \cdot \text{Conv Block Output}
$$

As $\alpha$ increases from 0 to 1, the output gradually transitions the conv block output, which ensures a smooth transition

The generator and discriminator grow symetrically, which means whenever a new layer is inserted into the generator, a new layer will be inserted into the discriminator too. However, the blending factor is not shared between generator and the discriminator, so they control the blending process independently. This is because sharing $\alpha$ could lead to instability. In general, the generator might need a longer blending period to adapt to a new resolution, while the discriminator might adapt more quickly. By keeping $\alpha$ separate, each model can transition at its own pace, and grow together until both models are ready